In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# 读取cvs训练数据集
cvs_path = "./data/csv/Molecular_Descriptor_train.csv"
df = pd.read_csv(cvs_path, index_col=0)
# 数据分割
from sklearn.model_selection import train_test_split

X = df.drop(['SMILES', 'pIC50'], axis=1)
y = df['pIC50']

In [3]:
# 去除sum为0的列和方差过小的列
sum_zero_count = 0
var_count = 0
delete = []
for index, row in df.iteritems():
    if index != "SMILES" and index != "pIC50":
        #计算该列和
        row = (row - row.min()) / (row.max() - row.min())

        if df[index].sum() == 0:
            df[index] = row
            delete.append((index))
            sum_zero_count = sum_zero_count + 1
            continue

        if row.var() < 0.005:
            r = row.var()
            var_count = var_count + 1
            delete.append((index))

final_sum = df.shape[1] - 1 - var_count - sum_zero_count
print("最终剩列", final_sum)
print("sum 0 列: ", sum_zero_count, "方差过小", var_count)
print(delete)
#删除sum为0后的数据
handle_df = df.drop(delete, axis=1).drop(["SMILES","pIC50"],axis=1)


最终剩列 333
sum 0 列:  225 方差过小 172
['ALogP', 'ALogp2', 'AMR', 'apol', 'nAtom', 'nHeavyAtom', 'nH', 'nB', 'nN', 'nP', 'nCl', 'nI', 'ATSc1', 'ATSc2', 'ATSc4', 'ATSc5', 'ATSm1', 'ATSm2', 'ATSm3', 'ATSm4', 'ATSm5', 'ATSp1', 'ATSp2', 'nBase', 'nBonds', 'nBonds2', 'nBondsS', 'nBondsS2', 'nBondsS3', 'nBondsD2', 'nBondsQ', 'bpol', 'C1SP2', 'VCH-4', 'SC-3', 'SC-6', 'SPC-4', 'SP-0', 'SP-1', 'SP-2', 'SP-3', 'VP-0', 'VP-1', 'VP-2', 'CrippenLogP', 'CrippenMR', 'ECCEN', 'nHBd', 'nwHBd', 'nHBa', 'nHBint2', 'nHBint3', 'nHBint5', 'nHBint6', 'nHBint7', 'nHBint8', 'nHBint9', 'nHBint10', 'nHdNH', 'nHsSH', 'nHsNH2', 'nHssNH', 'nHsNH3p', 'nHssNH2p', 'nHsssNHp', 'nHCsatu', 'nHmisc', 'nsLi', 'nssBe', 'nssssBem', 'nsBH2', 'nssBH', 'nsssB', 'nssssBm', 'nddC', 'ndssC', 'nsNH3p', 'nsNH2', 'nssNH2p', 'ndNH', 'nssNH', 'nsssNHp', 'nddsN', 'nssssNp', 'ndO', 'naOm', 'nsOm', 'nsSiH3', 'nssSiH2', 'nsssSiH', 'nssssSi', 'nsPH2', 'nssPH', 'nsssP', 'ndsssP', 'nddsP', 'nsssssP', 'nsSH', 'nssS', 'ndssS', 'nssssssS', 'nSm', 'nsCl

In [8]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
# lr = Ridge(alpha=100000, fit_intercept=True, normalize=True,
#            copy_X=True, max_iter=1500, tol=1e-4, solver='auto')

# lr = RandomForestRegressor()

lr = SVR(kernel="linear")
X_train, X_test, y_train, y_test = train_test_split(handle_df, y, test_size=0.2, random_state=0, shuffle=False)

scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.fit_transform(X_test)


# 这里不带交叉验证的部分
# estimator估计函数
rfe = RFE(estimator=lr, n_features_to_select=20,importance_getter=lr.coef_)
rfe.fit(X_test, y_test)
# 和传参对应，所选择的属性的个数
print(rfe.n_features_)
# 打印的是相应位置上属性的排名
print(rfe.ranking_)
# 属性选择的一种模糊表示，选择的是true，未选择的是false
print(rfe.support_)
# 外部估计函数的相关信息
print(rfe.estimator_)


AttributeError: 'SVR' object has no attribute '_dual_coef_'

In [10]:
pd.DataFrame(X_train.columns, index=rfe.ranking_, columns=['Rank']).sort_index(ascending=True).head(20)


,Rank
1,XLogP
1,SsOH
1,minHBa
1,minHBint4
1,minHBint6
1,VP-5
1,minHBint10
1,minHsOH
1,mindssC
1,VPC-6
